# 第07章：Embeddings & Vector Stores（嵌入与向量存储）

## 学习目标

本章将学习：
1. 理解Embeddings（嵌入）的概念和作用
2. 使用Embedding模型将文本转换为向量
3. 理解语义相似度和相似度度量
4. 掌握Vector Store的使用
5. 实现语义搜索功能

## 为什么需要Embeddings和Vector Stores？

在RAG系统中：
- Embeddings将文本转换为数字向量，捕获语义信息
- Vector Stores存储这些向量，支持快速的语义搜索
- 通过语义相似度而非关键词匹配来检索相关文档
- 这是RAG技术的核心基础

In [4]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import tool
from langchain.messages import HumanMessage, AIMessage, ToolMessage

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

# 配置Embedding模型（使用OpenAI兼容的API）
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL
)

print("环境配置完成！")
print(f"Chat模型: {model.model_name}")
print(f"Embedding模型: {embeddings.model}")

环境配置完成！
Chat模型: gpt-4.1-mini
Embedding模型: text-embedding-3-small


## 2. Embeddings（嵌入）

### 什么是Embeddings？

Embeddings是将文本转换为固定长度的数字向量的过程。这些向量捕获了文本的语义含义。

### 核心特点

1. **语义捕获**：相似含义的文本会产生相近的向量
2. **固定维度**：无论文本长短，都转换为相同维度的向量
3. **可计算**：可以用数学方法计算向量之间的相似度

### 为什么重要？

```
传统关键词搜索：
查询："机器学习"
只能匹配包含"机器学习"的文档

语义搜索（基于Embeddings）：
查询："机器学习"
可以匹配：
- "机器学习"（完全匹配）
- "深度学习"（相关概念）
- "神经网络"（相关概念）
- "AI算法"（相关领域）
```

In [5]:
### 2.1 生成单个文本的Embedding

print("【示例1：生成文本Embedding】")
print()

# 生成单个文本的embedding
text = "人工智能正在改变世界"
embedding = embeddings.embed_query(text)

print(f"原始文本: {text}")
print(f"向量维度: {len(embedding)}")
print(f"向量前10个元素: {embedding[:10]}")
print()
print("说明：文本被转换为1536维的向量（具体维度取决于模型）")

【示例1：生成文本Embedding】

原始文本: 人工智能正在改变世界
向量维度: 1536
向量前10个元素: [0.036390144377946854, 0.0006589797558262944, 0.04068123549222946, 0.05586833879351616, 0.033466313034296036, -0.03420253098011017, -0.012526202946901321, 0.07564102113246918, -0.007078198716044426, -0.01901542767882347]

说明：文本被转换为1536维的向量（具体维度取决于模型）


In [6]:
### 2.2 批量生成Embeddings

print("【示例2：批量生成Embeddings】")
print()

texts = [
    "机器学习是人工智能的一个分支",
    "深度学习使用多层神经网络",
    "今天天气很好，适合出门",
    "Python是一种编程语言"
]

# 批量生成embeddings（更高效）
batch_embeddings = embeddings.embed_documents(texts)

print(f"处理了 {len(texts)} 个文本")
print(f"每个向量的维度: {len(batch_embeddings[0])}")
print()
for i, text in enumerate(texts):
    print(f"文本 {i+1}: {text}")
    print(f"  向量前5个元素: {batch_embeddings[i][:5]}")
    print()

【示例2：批量生成Embeddings】

处理了 4 个文本
每个向量的维度: 1536

文本 1: 机器学习是人工智能的一个分支
  向量前5个元素: [0.021642746403813362, 0.003563135163858533, 0.0028191087767481804, -0.0396449975669384, 0.04252098873257637]

文本 2: 深度学习使用多层神经网络
  向量前5个元素: [-0.016283394768834114, -0.05267179757356644, 0.004675764590501785, -0.022784290835261345, 0.026668213307857513]

文本 3: 今天天气很好，适合出门
  向量前5个元素: [0.03830382227897644, 0.002759892027825117, -0.01888417638838291, -0.03285989537835121, 0.05365390703082085]

文本 4: Python是一种编程语言
  向量前5个元素: [0.009665939956903458, -0.029112696647644043, 0.0005882233381271362, -0.015951262786984444, 0.044801387935876846]



## 3. 相似度度量（Similarity Metrics）

### 常用的相似度度量方法

1. **余弦相似度（Cosine Similarity）**
   - 测量两个向量之间的角度
   - 值范围：-1到1（1表示完全相同，-1表示完全相反）
   - 最常用的方法

2. **欧氏距离（Euclidean Distance）**
   - 测量两个向量之间的直线距离
   - 值越小，相似度越高

3. **点积（Dot Product）**
   - 测量一个向量在另一个向量上的投影
   - 值越大，相似度越高

In [7]:
### 3.1 计算文本相似度

import numpy as np

print("【示例3：计算文本相似度】")
print()

# 定义余弦相似度函数
def cosine_similarity(vec1, vec2):
    """计算两个向量的余弦相似度"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

# 准备测试文本
query = "机器学习算法"
texts = [
    "深度学习是机器学习的子领域",    # 高度相关
    "今天天气很好",                   # 不相关
    "神经网络是深度学习的基础",      # 相关
    "我喜欢吃苹果"                    # 不相关
]

# 生成embeddings
query_embedding = embeddings.embed_query(query)
text_embeddings = embeddings.embed_documents(texts)

# 计算相似度
print(f"查询文本: {query}")
print()
print("相似度结果：")
similarities = []
for i, text in enumerate(texts):
    similarity = cosine_similarity(query_embedding, text_embeddings[i])
    similarities.append((text, similarity))
    print(f"{i+1}. {text}")
    print(f"   余弦相似度: {similarity:.4f}")
    print()

# 按相似度排序
similarities.sort(key=lambda x: x[1], reverse=True)
print("按相似度排序后：")
for i, (text, sim) in enumerate(similarities, 1):
    print(f"{i}. [{sim:.4f}] {text}")

【示例3：计算文本相似度】

查询文本: 机器学习算法

相似度结果：
1. 深度学习是机器学习的子领域
   余弦相似度: 0.5208

2. 今天天气很好
   余弦相似度: 0.1055

3. 神经网络是深度学习的基础
   余弦相似度: 0.3786

4. 我喜欢吃苹果
   余弦相似度: 0.1145

按相似度排序后：
1. [0.5208] 深度学习是机器学习的子领域
2. [0.3786] 神经网络是深度学习的基础
3. [0.1145] 我喜欢吃苹果
4. [0.1055] 今天天气很好


## 4. Vector Stores

### 什么是Vector Store？

Vector Store是专门用于存储和检索向量的数据库。

### 工作流程

索引阶段：文档 → Embedding模型 → 向量 → Vector Store

查询阶段：查询文本 → Embedding模型 → 查询向量 → 相似度搜索 → Top-K结果

### 常用的Vector Store

- InMemoryVectorStore：内存向量存储（开发测试）
- Chroma：开源向量数据库
- FAISS：Facebook的高性能向量检索库
- Pinecone：托管向量数据库服务
- Qdrant：高性能向量搜索引擎

In [8]:
### 4.1 创建Vector Store并添加文档

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document

print("【示例4：使用InMemoryVectorStore】")
print()

# 创建Vector Store
vector_store = InMemoryVectorStore(embeddings)

# 准备文档
documents = [
    Document(page_content="LangChain是一个用于构建LLM应用的框架", metadata={"source": "doc1", "topic": "框架"}),
    Document(page_content="RAG是检索增强生成技术", metadata={"source": "doc2", "topic": "技术"}),
    Document(page_content="向量数据库用于存储和检索向量", metadata={"source": "doc3", "topic": "数据库"}),
    Document(page_content="Embedding将文本转换为向量", metadata={"source": "doc4", "topic": "技术"}),
    Document(page_content="今天是个好天气", metadata={"source": "doc5", "topic": "天气"}),
]

# 添加文档到vector store
ids = vector_store.add_documents(documents)

print(f"成功添加 {len(documents)} 个文档")
print(f"文档IDs: {ids[:3]}...")
print()
print("文档已被自动转换为向量并存储！")

【示例4：使用InMemoryVectorStore】

成功添加 5 个文档
文档IDs: ['9b96c217-e1d1-4fb1-b7c0-5078215059cf', '8f761c5a-873c-4aad-8b62-e687edbb9b86', '679ae15c-6a0e-44f8-b315-fed47fbc44ab']...

文档已被自动转换为向量并存储！


In [9]:
### 4.2 相似度搜索

print("【示例5：相似度搜索】")
print()

# 搜索查询
query = "什么是RAG技术？"

# 执行相似度搜索（返回最相关的3个文档）
results = vector_store.similarity_search(query, k=3)

print(f"查询: {query}")
print()
print("搜索结果：")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")
    print(f"   元数据: {doc.metadata}")
    print()

【示例5：相似度搜索】

查询: 什么是RAG技术？

搜索结果：
1. RAG是检索增强生成技术
   元数据: {'source': 'doc2', 'topic': '技术'}

2. 向量数据库用于存储和检索向量
   元数据: {'source': 'doc3', 'topic': '数据库'}

3. LangChain是一个用于构建LLM应用的框架
   元数据: {'source': 'doc1', 'topic': '框架'}



In [10]:
### 4.3 带相似度分数的搜索

print("【示例6：带相似度分数的搜索】")
print()

query = "向量嵌入"

# 返回文档和相似度分数
results_with_scores = vector_store.similarity_search_with_score(query, k=3)

print(f"查询: {query}")
print()
print("搜索结果（带分数）：")
for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"{i}. [分数: {score:.4f}] {doc.page_content}")
    print(f"   元数据: {doc.metadata}")
    print()

print("注意：分数越低表示相似度越高（这是距离度量）")

【示例6：带相似度分数的搜索】

查询: 向量嵌入

搜索结果（带分数）：
1. [分数: 0.5969] 向量数据库用于存储和检索向量
   元数据: {'source': 'doc3', 'topic': '数据库'}

2. [分数: 0.5316] Embedding将文本转换为向量
   元数据: {'source': 'doc4', 'topic': '技术'}

3. [分数: 0.3213] RAG是检索增强生成技术
   元数据: {'source': 'doc2', 'topic': '技术'}

注意：分数越低表示相似度越高（这是距离度量）


In [ ]:
### 4.4 元数据过滤

print("【示例7：基于元数据的过滤搜索】")
print()

query = "技术"

# 定义过滤函数：只保留topic为"技术"的文档
def filter_func(doc: Document) -> bool:
    # 防止元数据中没有topic字段导致KeyError
    return doc.metadata.get("topic") == "技术"

# 只搜索topic为"技术"的文档
results = vector_store.similarity_search(
    query, 
    k=3,
    filter=filter_func  # 关键：传入函数而非字典
)

print(f"查询: {query}")
print(f"过滤条件: topic='技术'")
print()
print("搜索结果：")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")
    print(f"   元数据: {doc.metadata}")
    print()

print("只返回了符合过滤条件的文档！")

【示例7：基于元数据的过滤搜索】

查询: 技术
过滤条件: topic='技术'

搜索结果：
1. RAG是检索增强生成技术
   元数据: {'source': 'doc2', 'topic': '技术'}

2. Embedding将文本转换为向量
   元数据: {'source': 'doc4', 'topic': '技术'}

只返回了符合过滤条件的文档！


## 5. 完整流程：从文档处理到语义搜索

现在让我们结合第06章学到的文档处理知识，实现一个完整的语义搜索系统。

In [13]:
print("【完整流程演示】")
print()

# 步骤1：加载和分割文档
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("步骤1：加载文档")
loader = TextLoader("../data/langchain_intro.txt", encoding="utf-8")
docs = loader.load()
print(f"  加载了 {len(docs)} 个文档")

# 步骤2：分割文档
print("\n步骤2：分割文档")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)
splits = text_splitter.split_documents(docs)
print(f"  分割成 {len(splits)} 个块")

# 步骤3：创建Vector Store并索引
print("\n步骤3：创建向量存储并索引文档")
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(splits)
print(f"  已索引 {len(splits)} 个文档块")

# 步骤4：执行语义搜索
print("\n步骤4：语义搜索")
queries = [
    "LangChain的核心功能是什么？",
    "支持哪些模型提供商？"
]

for query in queries:
    print(f"\n查询: {query}")
    results = vector_store.similarity_search(query, k=2)
    for i, doc in enumerate(results, 1):
        print(f"  结果{i}: {doc.page_content[:80]}...")
        
print("\n完成！语义搜索系统已就绪。")

【完整流程演示】

步骤1：加载文档
  加载了 1 个文档

步骤2：分割文档
  分割成 3 个块

步骤3：创建向量存储并索引文档
  已索引 3 个文档块

步骤4：语义搜索

查询: LangChain的核心功能是什么？
  结果1: 2. 提示词管理
LangChain提供了强大的提示词模板系统，支持动态变量、条件逻辑和少样本学习。

3. 链式调用
通过LCEL（LangChain Exp...
  结果2: LangChain简介

LangChain是一个强大的框架，专门用于构建基于大型语言模型（LLM）的应用程序。它提供了一套完整的工具链，帮助开发者快速构建智能...

查询: 支持哪些模型提供商？
  结果1: LangChain简介

LangChain是一个强大的框架，专门用于构建基于大型语言模型（LLM）的应用程序。它提供了一套完整的工具链，帮助开发者快速构建智能...
  结果2: 5. Agent框架
LangChain提供了Agent框架，使模型能够自主决策和使用工具。

应用场景

- 问答系统
- 文档分析
- 代码生成
- 对话机...

完成！语义搜索系统已就绪。


## 6. 实战项目：智能文档问答系统

构建一个完整的文档问答系统，能够：
1. 索引多个文档
2. 理解自然语言查询
3. 返回相关文档片段
4. 使用LLM生成答案

In [14]:
print("【智能文档问答系统】")
print()

class DocumentQA:
    """智能文档问答系统"""
    
    def __init__(self, embeddings, llm):
        self.embeddings = embeddings
        self.llm = llm
        self.vector_store = InMemoryVectorStore(embeddings)
        self.doc_count = 0
    
    def index_documents(self, documents):
        """索引文档"""
        self.vector_store.add_documents(documents)
        self.doc_count += len(documents)
        return f"已索引 {len(documents)} 个文档，总计 {self.doc_count} 个"
    
    def search(self, query, k=3):
        """搜索相关文档"""
        return self.vector_store.similarity_search(query, k=k)
    
    def ask(self, question, k=3):
        """问答：检索相关文档并生成答案"""
        # 1. 检索相关文档
        relevant_docs = self.search(question, k=k)
        
        # 2. 构建上下文
        context = "\n\n".join([doc.page_content for doc in relevant_docs])
        
        # 3. 构建提示词
        prompt = f"""基于以下文档内容回答问题。如果文档中没有相关信息，请说"我不知道"。

文档内容：
{context}

问题：{question}

回答："""
        
        # 4. 生成答案
        response = self.llm.invoke(prompt)
        
        return {
            "question": question,
            "answer": response.content,
            "sources": relevant_docs
        }

# 创建问答系统
qa_system = DocumentQA(embeddings, model)

# 准备测试文档
test_docs = [
    Document(page_content="LangChain是一个开源框架，用于开发由语言模型驱动的应用程序。", 
             metadata={"source": "intro"}),
    Document(page_content="LangChain支持OpenAI、Anthropic、Google等多个模型提供商。", 
             metadata={"source": "models"}),
    Document(page_content="RAG（检索增强生成）是LangChain的核心应用场景，它结合了检索和生成。", 
             metadata={"source": "rag"}),
    Document(page_content="LangChain提供了Document Loaders、Text Splitters、Vector Stores等工具。", 
             metadata={"source": "tools"}),
]

# 索引文档
print(qa_system.index_documents(test_docs))
print()

# 测试问答
questions = [
    "LangChain是什么？",
    "LangChain支持哪些模型？",
    "什么是RAG？"
]

print("=" * 60)
for question in questions:
    result = qa_system.ask(question, k=2)
    print(f"\n问题: {result['question']}")
    print(f"答案: {result['answer']}")
    print(f"\n参考文档:")
    for i, doc in enumerate(result['sources'], 1):
        print(f"  {i}. {doc.page_content}")
    print("=" * 60)

print("\n智能问答系统演示完成！")

【智能文档问答系统】

已索引 4 个文档，总计 4 个


问题: LangChain是什么？
答案: LangChain是一个开源框架，用于开发由语言模型驱动的应用程序。

参考文档:
  1. LangChain是一个开源框架，用于开发由语言模型驱动的应用程序。
  2. LangChain提供了Document Loaders、Text Splitters、Vector Stores等工具。

问题: LangChain支持哪些模型？
答案: LangChain支持OpenAI、Anthropic、Google等多个模型提供商的模型。

参考文档:
  1. LangChain支持OpenAI、Anthropic、Google等多个模型提供商。
  2. LangChain是一个开源框架，用于开发由语言模型驱动的应用程序。

问题: 什么是RAG？
答案: RAG（检索增强生成）是LangChain的核心应用场景，它结合了检索和生成。

参考文档:
  1. RAG（检索增强生成）是LangChain的核心应用场景，它结合了检索和生成。
  2. LangChain是一个开源框架，用于开发由语言模型驱动的应用程序。

智能问答系统演示完成！


## 7. 总结与最佳实践

### 核心概念回顾

1. Embeddings：将文本转换为向量，捕获语义信息
2. Vector Stores：存储和检索向量
3. 相似度度量：余弦相似度、欧氏距离、点积
4. 语义搜索：基于含义而非关键词的搜索

### 关键要点

- 使用embed_query()处理查询，embed_documents()批量处理
- Vector Store提供add_documents()和similarity_search()方法
- 元数据过滤可以提高检索精度
- k值通常设置为3-5

### 最佳实践

1. 选择合适的Embedding模型和维度
2. 合理设置文档分割大小
3. 充分利用元数据
4. 根据应用选择合适的Vector Store

### 常见陷阱

- 索引和查询使用不同的embedding模型
- 文档太长没有分割
- k值设置不当
- 忽略元数据的作用